In [2]:
import csv

reviews=[]
for row in csv.reader(open('C:\\Users\\Hassan\\Desktop\\headline.csv', encoding = "utf-8")):
    reviews.append(row)
reviews = reviews[1:]  #This command removes the first row of the file as it only contains label(Heading)


In [3]:
import spacy
from spacy.lang.ur import Urdu
parser = Urdu()
from spacy.lang.ur.stop_words import STOP_WORDS
import nltk
stopwords = set(nltk.corpus.stopwords.words('urdu'))
#nltk.download('wordnet')
from nltk.corpus import wordnet as wn
import lemma

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.text.isdigit():
            lda_tokens.append('NUM')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    word = lemma.lemmatize(word)
    return word

final=[]     #In this list all of our pre processed data will be stored
for line in reviews:  #Break sentence into list words
    for words in line:
        tokens = tokenize(words)
        tokens = [token for token in tokens if len(token) > 3]
        tokens = [token for token in tokens if token not in stopwords]
        tokens = [get_lemma(token) for token in tokens]
        final.append(tokens)

In [4]:
from gensim import corpora
dictionary = corpora.Dictionary(final)
print(dictionary)

Dictionary(49655 unique tokens: ['بدبودار', 'سیارہ', 'شمس', 'ماہرین', 'نظامِ']...)


In [5]:
corpus = [dictionary.doc2bow(text) for text in final]

In [6]:
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)
ldamodel.show_topics()  

[(0,
  '0.063*"فلم" + 0.021*"بھارت" + 0.019*"حکومت" + 0.019*"پاکستان" + 0.017*"شکار" + 0.017*"کیلیا" + 0.016*"گلوکار" + 0.016*"کروڑ" + 0.013*"کمار" + 0.013*"گئ"'),
 (1,
  '0.032*"شاد" + 0.025*"کرینہ" + 0.022*"امراض" + 0.020*"کامیاب" + 0.017*"پاکستان" + 0.016*"بہترین" + 0.015*"سکتی" + 0.014*"دبنگ" + 0.011*"بھارت" + 0.011*"انداز"'),
 (2,
  '0.072*"کراچی" + 0.019*"ڈرامہ" + 0.018*"لاہور" + 0.017*"اسلام" + 0.015*"مکمل" + 0.015*"گلوکارہ" + 0.015*"حاصل" + 0.014*"ہیروئن" + 0.014*"پہنچ" + 0.014*"موسیق"'),
 (3,
  '0.042*"کپور" + 0.029*"فنکار" + 0.024*"علاج" + 0.020*"انتخاب" + 0.018*"زخم" + 0.017*"عامر" + 0.015*"واپس" + 0.011*"منائ" + 0.011*"رنبیر" + 0.011*"بلوچستان"'),
 (4,
  '0.050*"بھارت" + 0.039*"اداکارہ" + 0.032*"پاکستان" + 0.032*"کردار" + 0.018*"کترینہ" + 0.016*"وزیراعظم" + 0.016*"انسان" + 0.015*"ہلاک" + 0.015*"خلاف" + 0.015*"چوپڑا"'),
 (5,
  '0.055*"سلمان" + 0.048*"افراد" + 0.018*"مریض" + 0.018*"پولیس" + 0.015*"کمیشن" + 0.015*"بن" + 0.014*"بحال" + 0.013*"دوران" + 0.012*"تبدیل" + 0.011*"شوٹ

In [7]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.129598 -0.054809       1        1  12.418129
4      0.027251 -0.238539       2        1  11.461418
8     -0.203453 -0.007789       3        1  11.400841
3     -0.018898  0.016752       4        1  10.698031
0     -0.090923 -0.035108       5        1  10.668897
2      0.105006 -0.030546       6        1   9.418776
5     -0.056790  0.211566       7        1   9.270371
6     -0.076356  0.169292       8        1   9.202160
1      0.130855 -0.179780       9        1   8.017609
7      0.312907  0.148961      10        1   7.443768, topic_info=        Term         Freq        Total Category  logprob  loglift
472    کراچی  5034.000000  5034.000000  Default  30.0000  30.0000
1709     فلم  4976.000000  4976.000000  Default  29.0000  29.0000
4639   سلمان  3795.000000  3795.000000  Default  28.0000  28.0000
2632     بال  3465.000000  3465.000000  Default  27.0000  27.0000
549    بھارت  6669.000000  6669.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
217      پہل   949.741578  1306.660338  Topic10  -4.0685   2.2788
169     دوست   551.635852   690.401458  Topic10  -4.6119   2.3734
26225  دپیکا   437.681141   677.423048  Topic10  -4.8432   2.1610
1048   ناکام   409.039658   674.198915  Topic10  -4.9109   2.0981
24      تیار   374.853591  3208.108891  Topic10  -4.9982   0.4509

[432 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
19058      9  0.996113    آئٹم
3317       6  0.998601    آباد
3558       9  0.997402  آپریشن
1963       3  0.998234     اثر
90         3  0.997885   اجازت
...      ...       ...     ...
8674       6  0.997698    ہونگ
517        1  0.999087    ہوگئ
1865       2  0.759277   ہوگئا
1865       4  0.239485   ہوگئا
12758      6  0.998479  ہیروئن

[622 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 5, 9, 4, 1, 3, 6, 7, 2, 8])

In [16]:
lsimodel = LsiModel(corpus=corpus, num_topics=10, id2word=dictionary)
lsimodel.show_topics()

c:\users\hassan\appdata\local\programs\python\python38\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.928*"پاکستان" + 0.260*"بھارت" + 0.077*"کیلیا" + 0.059*"کرکٹ" + 0.043*"ورلڈ" + 0.041*"خلاف" + 0.040*"فیصلہ" + 0.040*"کیلئا" + 0.039*"کراچی" + 0.038*"حکومت"'),
 (1,
  '0.641*"کراچی" + 0.235*"افراد" + -0.205*"پاکستان" + 0.181*"سندھ" + 0.165*"گرفتار" + 0.165*"حکومت" + 0.162*"پولیس" + 0.162*"کیلیا" + 0.147*"بھارت" + 0.137*"سمیت"'),
 (2,
  '0.607*"بھارت" + -0.432*"کراچی" + -0.249*"پاکستان" + 0.244*"حکومت" + 0.223*"کیلیا" + 0.160*"وزیراعظم" + -0.114*"مارکیٹ" + 0.109*"فیصلہ" + 0.109*"عمران" + -0.102*"افراد"'),
 (3,
  '-0.691*"بھارت" + 0.280*"کیلیا" + 0.280*"حکومت" + 0.178*"سندھ" + -0.163*"کراچی" + 0.143*"وزیراعظم" + 0.136*"کورٹ" + 0.136*"عمران" + 0.128*"فیصلہ" + 0.116*"سپریم"'),
 (4,
  '0.796*"کیلیا" + 0.190*"کراچی" + -0.178*"افراد" + -0.168*"حکومت" + -0.157*"عمران" + -0.152*"دہشت" + 0.113*"تیار" + -0.110*"شریف" + -0.105*"گرد" + 0.104*"مارکیٹ"'),
 (5,
  '0.460*"افراد" + -0.375*"کراچی" + 0.329*"کیلیا" + 0.292*"دہشت" + 0.241*"زخم" + -0.214*"مارکیٹ" + -0.207*"وزیراعظم" + 0.187*"ہلاک" + 

In [ ]:
!python -m pip install -U pyLDAvis